<font size="6">Рекуррентные нейронные сети (RNN)</font>

#  Особенности рекуррентных нейронных сетей

До этого мы работали с методами машинного обучения, которые работают с признаковым описанием фиксированной длины.

Так правильно делать в случае табличных данных &mdash; обычно каждый объект в таблицах описан фиксированным набором признаков, и новые признаки не могут появиться "вдруг". Мы точно знаем, что на вход нам приходит объект размера 100, а на выходе мы должны для него предсказать 1 число. 

Верно это и про данные в виде изображений &mdash; обычно наша нейросеть учится на изображениях определенного разрешения. Да, иногда мы делаем нейросеть, которая способна работать с изображением почти любого разрешения, но добиваемся мы этого за счет вставки слоев global pooling, которые просто приводят любое признаковое описание, полученное нашей нейросетью, а точнее, ее сверточной частью, ранее, к фиксированному размеру. 

Однако многие данные структурой фиксированной длины не обладают. К примеру &mdash; тексты. Возьмем все абзацы из "Войны и Мира". Какие-то будут больше, какие-то меньше. И обрезать их как-то нельзя. Аналогично будет и для текстов из Твиттера и т.д. И что делать, если мы хотим предсказывать, например, эмоциональную окрашенность текста? 

Более того, а что делать, если, к примеру, на основе абзаца текста нам необходимо сгенерировать его краткое содержание? То есть и то, что нужно предсказать, у нас может быть разной длины. Аналогично &mdash; мы хотим по данным о курсе валюты за прошлый год спрогнозировать курс валюты на следующий месяц по дням.  

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/time_series_data.png" width="700">

Таким образом, у нас есть целый набор задач, для которых необходима особая структура нейросети, позволяющая, так или иначе, принимать данные разного размера. 



## Примеры задач

**Анализ временных рядов**
- Табличные данные
- Аннотирование изображений и видео (Image/Video captioning)
- Машинный перевод
- Распознавание текста
- Распознавание речи


**Генеративные модели**
- Генерация текста/речи (например, чат - боты)
- Генерация изображений

**Классификация**
- Изображения
- Блоки текста (Sentiment analysis)

**Анализ последовательностей**
- анализ текстов
- биологические последовательности 
- химические последовательности


Общее для задач - мы имеем возможность сохранять информацию, сформированную при обработке одной части объекта (**токена**), и использовать ее, когда мы анализируем другие части. 

До прихода нейронных сетей предложения пытались описать при помощи набора правил - грамматик, которые довольно успешно могли генерировать новые осмысленные предложения. Так выглядит разбор при помощи грамматик простого предложения 


<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/parse_tree.png" width="500">

Однако грамматики плохо учитывали более глобальные связи. Например, очень тяжело было добиться того, чтобы в абзаце текста:

"Леша пришел домой. Он будет есть рыбу"

Компьютер понял, что во втором предложении "Он" соответствует "Леше". 


Основная идея, на которой основано RNN, состоит в следующем &mdash; а давайте возьмем и всю последовательность пропустим через одну и туже нейросеть. 
Но при этом сама нейросеть кроме следующего элемента последовательности (например, слова в тексте), будет принимать еще один параметр &mdash; некий $v$, который в начале будет, например, вектором из нулей, а далее &mdash; значением, которое выдает сама нейросеть после обработки очередного элемента последовательности (**токена**)

Также далее мы будем использовать понятие **нулевого токена** - токена, который символизирует заплатку, токен, не несущий никакого смысла, но который иногда нужно передать модели. Например - как сигнал начала работы

В этом $v$ (чаще обозначается $h$) &mdash; будет храниться hidden state, состояние, учитывающее и локальный, и глобальный контекст.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/rnn_idea.png" width="800">


[Recurrent neural network
](https://en.wikipedia.org/wiki/Recurrent_neural_network)

При этом наша нейросеть может выдавать некий ответ на каждом шаге, но мы можем:

 1. Использовать только выданное на последнем (если нам нужно предсказать одно значение) &mdash; many-to-one

 2. Мы можем подавать в наше нейросетку токены (когда кончился исходный сигнал &mdash; подаем нулевые токены), пока она не сгенерирует токен, символизирующий остановку (many-to-many, one-to-many)

 3. Можем делать различные комбинации, игнорируя выходы нейросети в начале, например

«One to one» &mdash; обычная нейронная сеть, RNN здесь не нужно

Более сложной является реализация «one to many», когда у нас есть всего один вход и нам необходимо сформировать несколько выходов. Такой тип нейронной сети актуален, когда мы говорим о генерации музыки или текстов. Мы задаем начальное слово или начальный звук, а дальше модель начинает самостоятельно генерировать выходы, в качестве входа к очередной ячейке рассматривая выход с прошлой ячейки нейронной сети.

 Если мы рассматриваем задачу классификации, то актуальна схема «many to one». Мы должны проанализировать все входы нейронной сети и только в конце определиться с классом.
 
  Схему «many to many», когда количество выходов равно количеству входов нейронной сети. Обычно это задачи вида: разметить исходную последовательность. Например - указать столицы городов, названия важных объектов, веществ и т.д., что относится к задачам вида NER (Named entity recogition).
  
Ну и последней разновидностью нейронных сетей является сеть вида «many to many», когда количество выходов нейронной сети не равно количеству входов. Это актуально, к примеру, в машинном переводе, когда одна и та же фраза может иметь разное количество слов в разных языках (т.е. это реализует схему кодировщик-декодировщик). Кодировщик получает данные различной длины &mdash; например, предложение на английском языке. С помощью скрытых состояний он формирует из исходных данных вектор, который затем передаётся в декодировщик. Последний, в свою очередь, генерирует из полученного вектора выходные данные &mdash; исходную фразу, переведённую на другой язык.

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/one_or_many_to_one_or_many_ways.png" width="1000">

Можно объединять разные подходы. Сначала генерируем некий $h$, который содержит сжатую информацию о том, что было подано в нейросеть, а затем подаем его в нейросеть «one to many», которая генерирует, к примеру, перевод того текста, что был подан первой части нейросети. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/sequence_to_sequence.png" width="900">

# Базовый RNN блок

Рассмотрим работу рекуррентной нейронной сети:
1. На вход нейронной сети поступает некоторая последовательность $x = \{x_1,...x_t,...,x_n\}$. 

2. Для каждого поступившего $x_t$ мы формируем скрытое состояние $h_t$, которое является функцией от предыдущего состояния $h_{t-1}$ и текущего элемента последовательности $x_t$:
$$h_t = f_W(h_{t-1}, x_t),$$
где $W$  - это обучаемые параметры (веса).

3. На основании рассчитанного скрытого состояния, учитывающего предыдущие значения  $x_i$ формируется выходная последовательность $y = \{y_1,...y_t,...,y_k\}$. Для формирования предсказания $y_t$ в текущий момент времени в модель могут быть добавлены полносвязные слои, принимающие на вход текущее скрытое состояние $h_t$.

Ниже представлена простая RNN. В качестве функции активации используется тангенс.  

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/rnn_basic_block.png" width="700">

Важное отличие от слоев, с которыми мы уже сталкивались, состоит в том, что на выходе мы получаем два объекта: $y_t$ и $h_t$.

$y_t$ &mdash; предсказание в текущий момент времени, например метка класса.

$h_t$ &mdash; контекст, в котором предсказание было сделано. Он может использоваться для дальнейших предсказаний.

## RNNCell

В Pytorch для вычисления $h_t$ используется модуль [RNNCell](https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html)  

$y_t$ в нем не вычисляется, предполагается, что для его получения в модель должен быть добавлен дополнительный линейный слой.

**`input_size`** &mdash;  размер элемента последовательности.

В отличие от сверточных, это всегда вектор, а не тензор, поэтому **`input_size`** &mdash;  скаляр.

**`hidden_size`** &mdash; тоже скаляр. Он задает размер скрытого состояния, которое тоже является вектором. Фактически это количество нейронов в слое.


In [ ]:
import torch

rnn_cell = torch.nn.RNNCell(input_size=3, hidden_size=2)
dummy_sequence = torch.randn((1, 3)) # batch, input_size
h = rnn_cell(dummy_sequence) 
print("Inital shape:".ljust(17), f"{dummy_sequence.shape}")
print("Resulting shape:".ljust(17), f"{h.shape}")  # hidden state 

Внутри происходит примерно следующее:  
Для понятности в данном примере опущена батчевая обработка.

In [ ]:
import numpy as np
from torch import nn

# Simple RNNcell without a bias and batch support
class SimplifiedRNNCell(nn.Module): 
  def __init__(self, input_size, hidden_size):
    super().__init__()
    # Init weight matrix, for simplicity omit bias
    self.W_hx = np.random.randn(input_size, hidden_size) * 0.0001 # hidden_size == number of neurons
    self.W_hh = np.random.randn(hidden_size, hidden_size) * 0.0001 # naive initialization
    self.h0 = np.zeros((hidden_size)) # Initial hidden state
  
  def forward(self, x, h=None): # Without a batch dimension
    if h is None:
      h = self.h0
    h = np.tanh(self.W_hx.T.dot(x) + self.W_hh.T.dot(h))
    return h 
  
simple_rnn_cell = SimplifiedRNNCell(input_size=3, hidden_size=2)
h = simple_rnn_cell(dummy_sequence[0]) # No batch 
print(f"Out = h\n{h.shape} \n{h}")


Однако в последовательности всегда несколько элементов. И надо применить алгоритм к каждому.


 Поэтому RNNCell напрямую не используется. Для него есть обертка: [RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html) который обеспечивает последовательный вызов RNNCell для всех элементов последовательности.

### RNN блок в Pytorch 

**Warning: batch dim is second!**

In [ ]:
rnn = torch.nn.RNN(input_size=3, hidden_size=2) # batch_first = True
dummy_batched_seq = torch.randn((2, 1, 3)) # seq_len, batch, input_size
out, h = rnn(dummy_batched_seq) 

print("Inital shape:".ljust(20), f"{dummy_batched_seq.shape}")
print("Resulting shape:".ljust(20), f"{out.shape}")
print("Hidden state shape:".ljust(20), f"{h.shape}")

Внутри происходит примерно следующее:

In [ ]:
# Simple RNN without batching

class SimplifiedRNNLayer(nn.Module): 
  def __init__(self, input_size, hidden_size):
    super().__init__()
    self.rnn_cell = SimplifiedRNNCell(input_size, hidden_size)

  # Without a batch dimension x have shape seq_len * input_size
  def forward(self, x, h=None):
    all_h = []
    for i in range(x.shape[0]): # iterating over timestamps
      h = self.rnn_cell(x[i], h)
      all_h.append(h) 
    return  np.stack(all_h), h

simple_rnn = SimplifiedRNNLayer(input_size=4, hidden_size=2)
sequence = np.array([[0, 1, 2, 0], [3, 4, 5, 0]]) # batch with one sequence of two elements 

out, h = simple_rnn(sequence)
print("Inital shape:".ljust(20), f"{sequence.shape}")
print("Resulting shape:".ljust(20), f"{out.shape}")
print("Hidden state shape:".ljust(20), f"{h.shape}")

Давайте разберемся.

К данным добавляется еще одно измерение размер последовательности. Таким образом, batch из 5 последовательностей по 6 элементов в каждой будет выглядеть так (время идёт первой размерностью, потому поэлементно идём "сверху вниз"):

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/rnn_batch.png" width="600">

P.S. Размер самого элемента == 3

Внутри RNN модуля элементы последовательности обрабатываются последовательно:

<img src = "https://edunet.kea.su/repo/EduNet-content/L08/out/simple_rnn_h_state.png" width="700">

Веса при этом используются одни и те же.

In [ ]:
dummy_seq = torch.randn((2, 1, 3)) #  seq_len, batch, input_size

print("RNNCell")
rnn_cell = torch.nn.RNNCell(3, 2)
print("Parameter".ljust(10), "Shape")
for t, p in rnn_cell.named_parameters():
  print(t.ljust(10), p.shape) 

cell_out = rnn_cell(dummy_seq[0, :, :]) # take first element from sequence 
print()
print("Result shape =".ljust(20), cell_out.shape)
print("Hidden state shape =".ljust(20), cell_out.shape) # one hidden state

print('----------------------------------------')

print("RNN")
rnn = torch.nn.RNN(3, 2)
print("Parameter".ljust(15), "Shape")
for t, p in rnn.named_parameters():
  print(t.ljust(15), p.shape) 

out, h = rnn(dummy_seq)

print()
print("Result shape =".ljust(20), out.shape) # h for all timestamps element
print("Hidden state shape =".ljust(20), cell_out.shape) # h for last element

### Слои (Stacked RNNs)


RNN блоки можно объединять в слои, накладывая их друг на друга. Для этой операции в `torch.nn.RNN` есть аргумент `num_layers`, с помощью которого можно указать количество слоёв.

<img src ="http://edunet.kea.su/repo/EduNet-content/L08/out/layers.png" width="600">

In [ ]:
dummy_input = torch.randn((2, 1, 3))  #seq_len, batch, input_size
rnn = torch.nn.RNN(3, 2, num_layers=3)

# Weights matrix sizes not changed!
for t, p in rnn.named_parameters():
    print(t, p.shape) 

out, h = rnn(dummy_input)

print("Out:\n", out.shape) # Hidden states for all elements from top layer
print("h:\n", h.shape) # Hidden states for last element for all layers

### Bidirectional

Последовательность можно пропустить через сеть два раза: в прямом и обратном направлении. Для этого создаётся слой, аналогичный входному, для обратного направления, и результат двух слоёв конкатинируется.

[Understanding RNN implementation in PyTorch](https://medium.com/analytics-vidhya/understanding-rnn-implementation-in-pytorch-eefdfdb4afdb)

<img src ="http://edunet.kea.su/repo/EduNet-content/L08/out/bidirectional.png" width="700">

In [ ]:
dummy_input = torch.randn((2, 1, 3))  #seq_len, batch, input_size
rnn = torch.nn.RNN(3, 2, bidirectional=True)

for t, p in rnn.named_parameters():
    print(t, p.shape) 

out, h = rnn(dummy_input)

print("Out:\n", out.shape) # Concatenated Hidden states from both layers
print("h:\n", h.shape) # Hidden states last element from  both : 2*num_layers*hidden_state

# Пример прогнозирования временного ряда

---
- [How to Remove Non-Stationarity in Time Series Forecasting](https://towardsdatascience.com/how-to-remove-non-stationarity-in-time-series-forecasting-563c05c4bfc7)
- [A Guide to Time Series Forecasting in Python](https://builtin.com/data-science/time-series-forecasting-python)
- [Detecting stationarity in time series data](https://towardsdatascience.com/detecting-stationarity-in-time-series-data-d29e0a21e638)
- [Most useful Python functions for Time Series Analysis](https://towardsdatascience.com/most-useful-python-functions-for-time-series-analysis-ed1a9cb3aa8b)
- [Data transformations and forecasting models: what to use and when](https://people.duke.edu/~rnau/whatuse.htm)

Что общего у прогнозирования потребления электроэнергии домохозяйствами, оценки трафика на дорогах в определенные периоды, прогнозировании паводков и прогнозировании цены, по которой акции будут торговаться на фондовой бирже?

Все они подпадают под понятие данных временных рядов! Вы не можете точно предсказать любой из этих результатов без компонента «время». И по мере того, как в мире вокруг нас генерируется все больше и больше данных, прогнозирование временных рядов становится все более важной областью применения методов ML и DL.



## Подготовка данных 
[Time Series Prediction with LSTM Using PyTorch](https://colab.research.google.com/github/dlmacedo/starter-academic/blob/master/content/courses/deeplearning/notebooks/pytorch/Time_Series_Prediction_with_LSTM_Using_PyTorch.ipynb#scrollTo=NabsV8O5BBd5https://colab.research.google.com/github/dlmacedo/starter-academic/blob/master/content/courses/deeplearning/notebooks/pytorch/Time_Series_Prediction_with_LSTM_Using_PyTorch.ipynb#scrollTo=NabsV8O5BBd5)


### Загрузка данных

Air Passengers per month.  
[Dataset Air Passengers Number of air passengers per month](https://www.kaggle.com/rakannimer/air-passengers)


In [ ]:
import pandas as pd
dataset = pd.read_csv('https://edunet.kea.su/repo/EduNet-web_dependencies/L08/airline-passengers.csv')
dataset.head()

In [ ]:
import matplotlib.pyplot as plt

training_data = dataset.iloc[:, 1:2].values # transform dataframe to numpy.array
# plotting
plt.figure(figsize=(12, 4))
plt.plot(training_data, label='Airline Passangers Data')
plt.title("Number of passengers per month")
plt.ylabel("#passengers")
plt.xlabel("Month")
labels_to_display = [i for i in range(training_data.shape[0]) if i % 12 == 0]
plt.xticks(labels_to_display, dataset['Month'][labels_to_display])
plt.grid()
plt.show()

### Шкалирование данных

In [ ]:
# Min - Max normalization
td_min = training_data.min()
td_max = training_data.max()
print('Initial statistics:')
print('Minimum value:', repr(td_min).rjust(5))
print('Maximum value:', repr(td_max).rjust(5))

training_data = (training_data - td_min) / (td_max - td_min)
print('\nResulting statistics:')
print('Minimum value:', repr(training_data.min()).rjust(5))
print('Maximum value:', repr(training_data.max()).rjust(5))

### Формирование ансамблей данных

Поскольку мы хотим научиться предсказывать следующие значение на основе предыдущих, нам нужно подготовить данные соответствующим образом.

Разобьем весь массив данных на фрагменты вида

$x \to y$

где $x$ - это подпоследовательность, например записи с 1-й по 8-ю, а $y$ - это значение из 9-й записи, то самое, которое мы хотим предсказать.

In [ ]:
import numpy as np
import torch

# create data "ensemble"
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data) - seq_length):
        _x = data[i:(i + seq_length)] # picking several sequential observations
        _y = data[i+seq_length] # picking the subsequent observation
        x.append(_x)
        y.append(_y)

    return torch.Tensor(np.array(x)), torch.Tensor(np.array(y))
    
# set length of the ensemble; accuracy of the predictions and 
# speed perfomance almost always depend on it size
seq_length = 8 # compare 2 and 32
x, y = sliding_windows(training_data, seq_length)
print("Example of the obtained data:\n")
print("Data corresponding to the first x:")
print(x[0])
print("Data corresponding to the first y:")
print(y[0])

Благодаря такому подходу, мы можем работать с RNN моделью так же, как работали со сверточными моделями. Подавая на вход такую подпоследовательность + результат.

### Разобьем на train и test

In [ ]:
train_size = int(len(y) * 0.8)

x_train = x[:train_size]
y_train = y[:train_size]

x_test = x[train_size:]
y_test = y[train_size:]

print("Train data:")
print("x shape:", x_train.shape)
print("y shape:", y_train.shape)

print("\nTest data:")
print("x shape:", x_test.shape)
print("y shape:", y_test.shape)

## Создание и обучение модели

In [ ]:
import torch.nn as nn

class AirTrafficPredictor(nn.Module):

    def __init__(self, input_size, hidden_size):
        # hidden_size == number of neurons 
        super().__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1) # Predict only one value

    def forward(self, x):
        #print("x: ",x.shape) # 108 x 8 x 1 : [batch_size, seq_len, input_size] 
        out, h = self.rnn(x) 
        #print("out: ", out.shape) # 108 x 8 x 4 : [batch_size, seq_len, hidden_size] Useless!
        #print("h : ", h.shape) # 1 x 108 x 4 [ num_layers, batch_size, hidden_size]
        y = self.fc(h)
        #print("y",y.shape) # 1 x 108 x 1
        return y, h

### Обучение

Благодаря подготовке данных процесс обучения не будет отличаться от того что мы использовали на прошедших занятиях.

В силу того, что датасет маленький, все данные поместились в один batch, итерирования по batch-ам в явном виде здесь не происходит.

In [ ]:
def time_series_train(model, num_epochs=2000, learning_rate=0.01):
  
  criterion = torch.nn.MSELoss() # mean-squared error for regression
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # Train the model
  for epoch in range(num_epochs):
      y_pred, h = model(x_train) # we don't use h there, but we can!
      optimizer.zero_grad()
      
      # obtain the loss
      loss = criterion(y_pred[0], y_train) # for shape compatibility
      loss.backward()
      
      optimizer.step()
      if epoch % 100 == 0:
          print(f"Epoch: {epoch},".ljust(15), "loss: %1.5f" % (loss.item()))

print("Simple RNN training process with MSE loss:")
input_size = 1
hidden_size = 4 
rnn = AirTrafficPredictor(input_size, hidden_size)
time_series_train(rnn)

### Тестирование

In [ ]:
def time_series_plot(train_predict):
  data_predict = train_predict.data
  y_data_plot = y.data

  # Denormalize
  data_predict = data_predict[0] * (td_max - td_min) + td_min
  y_data_plot = y_data_plot * (td_max - td_min) + td_min 

  # Ploitting
  plt.figure(figsize=(12, 4))
  plt.axvline(x=train_size, c='r', linestyle='--')
  # shifting the curve as first y-value not correspond first value overall
  plt.plot(seq_length + np.arange(y_data_plot.shape[0]), y_data_plot)
  plt.plot(seq_length + np.arange(y_data_plot.shape[0]), data_predict)
  
  plt.title("Number of passengers per month")
  plt.ylabel("#passengers")
  plt.xlabel("Month")
  plt.xticks(labels_to_display, dataset['Month'][labels_to_display])
  
  plt.legend(['Train/Test separation', 'Real', 'Predicted'])
  plt.grid(axis='x')
  plt.show()

rnn.eval()
train_predict, h = rnn(x)
time_series_plot(train_predict)

Видим, что модель в принципе справляется с задачей. Но посмотрим, как далее справятся более мощные архитектуры, и уже позднее напишем окончательные выводы

# Пример посимвольной генерации текста

[Github RNN-walkthrough](https://github.com/gabrielloye/RNN-walkthrough/blob/master/main.ipynb)

Одним из основных направлений использования рекуррентных сетей является работа с текстами:
- генерация (Language modeling)  
и 
- перевод (Machine Translation)

Давайте посмотрим, как решаются такого рода задачи.

Начнем с относительно простой - посимвольной генерации текста.

**Постановка задачи:**

предсказать следующий символ в последовательности.

- исходный текст:  
'hey how are you'

- искаженный текст:  
'hey how are yo'

- Верное предсказание:
'u'


Теоретически эту технику можно использовать для генерации подсказок при наборе текстов, исправления ошибок или восстановления частично утраченного текста.





<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/character_by_character_generation_example.gif" width="400">

[The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

## Подготовка данных

1. Зафиксировать словарь
2. Разбить данные
3. Выполнить кодирование символов 

In [ ]:
import pprint

text = ['hey how are you','good i am fine','have a nice day']

# Join all the sentences together and extract the unique characters 
# from the combined sentences
chars = set(''.join(text))

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}

print("Dictionary for mapping character to the integer:")
pprint.pprint(char2int)

Вместо ASCII символа, каждой букве мы сопоставили номер.

### Выравнивание данных (Padding)

RNN допускают работу с данными переменной длины. Но чтобы поместить предложения в batch надо их выровнять.


Обычно размер батча делают равным самому длинному предложению, а остальные просто дополняют пробелами до этого размера.

In [ ]:
lengths = [len(sent) for sent in text]
maxlen = max(lengths)
print(f"The longest string has {maxlen} characters.\n")

print(f"Initial texts:\n{text}")
# A simple loop that loops through the list of sentences and adds
# a ' ' whitespace until the length of the sentence matches
# the length of the longest sentence
for i in range(len(text)):
    while len(text[i])<maxlen:
        text[i] += ' '

print(f'Resulting texts:\n{text}')

### Разбиение данных



В качестве входа будем использовать предложение без последнего символа:

**'hey how are yo'**

, а в качестве результата - предложение, в котором он сгенерирован.

**'ey how are you'**


In [ ]:
# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []

for i in range(len(text)):
    # Remove last character for input sequence
    input_seq.append(text[i][:-1])
    
    # Remove firsts character for target sequence
    target_seq.append(text[i][1:])
    
    print("Input sequence:".ljust(18), f"'{input_seq[i]}'")
    print("Target sequence:".ljust(18), f"'{target_seq[i]}'")
    print()

### Кодирование

Теперь символы надо перевести в числа. Для этого мы уже построили словарь.

P.S. Запускать блок только один раз.

In [ ]:
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

    print("Encodded input sequence:".ljust(25), input_seq[i])
    print("Encodded target sequence:".ljust(25), target_seq[i])
    print()

#### One-hot encoding

Теперь из чисел надо сделать вектора. 


Почему бы не оставить числа?
В прошлом примере модель хорошо с ними работала.

В прошлом примере использовался MSE, и на выходе было число.

Если бы мы определили отношение порядка над номерами букв, то что-то подобное можно было бы сделать.

Однако сейчас мы предсказываем класс буквы.
Поэтому на входе и на выходе должен быть вектор.

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/one_hot_encoding_softmax.png" width="250">

In [ ]:
import numpy as np

dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
print("Input shape: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))
print(input_seq[0])

Каждый символ закодировали вектором.
Не слишком экономно, зато удобно умножать на матрицу весов.

P.S. Запускать только один раз

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/language_modeling.png" width="700">

In [ ]:
# Convert data to tensor
import torch

input_seq = torch.Tensor(input_seq)
target_seq = torch.Tensor(target_seq)

## Создание и обучение модели

In [ ]:
import torch.nn as nn
class NextCharacterGenerator(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super().__init__()

        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_size=hidden_dim, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        batch_size = x.size(0)
        #Initializing hidden state for first input using method defined below
        hidden_0 = torch.zeros(1, batch_size, self.rnn.hidden_size) # 1 correspond to number of layers

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden_0)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        # Need Only if n_layers > 1
        out = out.contiguous().view(-1, self.rnn.hidden_size)
        out = self.fc(out)
        
        return out, hidden

### Обучение

In [ ]:
# Instantiate the model with hyperparameters
model = NextCharacterGenerator(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)

# Define hyperparameters
num_epochs = 100

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training Run
for epoch in range(1, num_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    output, hidden = model(input_seq)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch % 10 == 0:
        print(f'Epoch: {epoch}/{num_epochs}'.ljust(20), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

### Тест

In [ ]:
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    
    out, hidden = model(character)
    #print(out.shape)
    #print(out)
    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for _ in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

sample(model, 15, 'good')

### Проблемы RNN

Теоретически, можно было бы сразу пропустить все данные через сеть и затем вычислить градиент, однако возникнут следующие проблемы:

 - Большие последовательности не поместятся в памяти
 - Поскольку цепочка будет очень длинной, то возникнет исчезновение/взрыв градиента
 - По мере прохождения сигнала по цепи контекст затирается


Исчезающий/взрывающийся градиент (Vanishing/exploding gradient) - явления исчезающего и взрывающегося градиента часто встречаются в контексте RNN. И при большой длине последовательности это становится критичным. Причина в том, что зависимость величины градиента от числа слоёв экспоненциальная, поскольку веса умножаются многократно.

$dL ∝ (W)^N$.

$W > 1$ => взрыв

$W < 1$ => затухание

<img src ="http://edunet.kea.su/repo/EduNet-content/L08/out/498_FA2019_lecture12-086.png" >

Один из путей решения проблемы: градиентное отсечение (Gradient clipping) - метод, который ограничивает максимально допустимое значение градиента, позволяя избежать градиентного взрыва.

# LSTM



Обычная RNN имело множество проблем, в том числе, в нем очень быстро затухала информация о предыдущих словах в предложении. Помимо этого, были проблемы с затуханием/взрывом самого градиента. 

Эти проблемы были частично решены в LSTM, предложенной в [Hochreiter & Schmidhuber (1997)](http://www.bioinf.jku.at/publications/older/2604.pdf)

В обычной RNN в ячейке был только один путь передачи информации. На каждом шаге мы сливали информацию, накопленную с предыдущих шагов, с текущей:


<img src = "https://edunet.kea.su/repo/EduNet-content/L08/out/simple_rnn_h_state.png" width="700px">


При этом информация о предыдущих токенах очень быстро затухает и теряется общая информация о предложении

Структура ячейки LSTM намного сложнее. Здесь есть целых 4 линейных слоя, каждый из которых выполняет разные задачи.

<img src="https://edunet.kea.su/repo/EduNet-content/L08/out/lstm_chain.png" width="700px">

<img src="https://edunet.kea.su/repo/EduNet-content/L08/out/lstm_chain_notation.png" width="700px">

Главное нововведение - в LSTM же добавлен путь $c$, который по задумке должен этот общий контекст сохранять. 

<img src="https://edunet.kea.su/repo/EduNet-content/L08/out/lstm_c_state_highway.png" width="300px">

Другими словами, путь $c$ (иногда называется highway, магистраль)  помогает нейросети сохранять важную информацию, встретившуюся в какой-то момент в предложении, все время, пока эта информация требуются. 



## LSTMCell

[pytorch LSTMCELL](https://pytorch.org/docs/stable/generated/torch.nn.LSTMCell.html)

Интерфейс отличается от RNNCell количеством входов и выходов


In [ ]:
import torch

lstm_cell = torch.nn.LSTMCell(input_size=3, hidden_size=4)
input = torch.randn(1, 3) # batch, input_size
h_0 = torch.randn(1, 4)
c_0 = torch.randn(1, 4)
h, c = lstm_cell(input, (h_0, c_0)) # second arg is tuple
print("Shape of h:", h.shape) # batch, hidden_size
print("Shape of c:", c.shape) # batch, hidden_size

## LSTM in Pytorch

Отличие от RNN состоит в том, что кроме $h$ возвращается еще и $c$. 

In [ ]:
import torch.nn as nn

lstm = nn.LSTM(input_size=4, hidden_size=5)
input = torch.randn(3, 2, 4) # seq_len, batch, input_size
out, (h, c) = lstm(input) # h and c returned in tuple

print("Input shape:".ljust(15), input.shape)
print("Shape of h".ljust(15), h.shape) # batch, hidden_size
print("Shape of c".ljust(15), c.shape) # batch, hidden_size
print("Output shape:".ljust(15), out.shape) # seq_len, batch, hidden_size : h for each element

### Пример использования на задаче с временным рядом

Чтобы убедиться в работоспособности конструкции, заменим RNN блок на LSTM в задаче предсказания временного ряда.

In [ ]:
import pandas as pd
dataset = pd.read_csv('https://edunet.kea.su/repo/EduNet-web_dependencies/L08/airline-passengers.csv')
dataset.head()

In [ ]:
# Min - Max normalization
training_data = dataset.iloc[:, 1:2].values
td_min = training_data.min()
td_max = training_data.max()
print('Initial statistics:')
print('Minimum value:', repr(td_min).rjust(5))
print('Maximum value:', repr(td_max).rjust(5))

training_data = (training_data - td_min) / (td_max - td_min)
print('\nResulting statistics:')
print('Minimum value:', repr(training_data.min()).rjust(5))
print('Maximum value:', repr(training_data.max()).rjust(5))

In [ ]:
import numpy as np

# create data "ensemble"
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data) - seq_length):
        _x = data[i:(i + seq_length)] # picking several sequential observations
        _y = data[i + seq_length] # picking the subsequent observation
        x.append(_x)
        y.append(_y)

    return torch.Tensor(np.array(x)), torch.Tensor(np.array(y))
    
# set length of the ensemble; accuracy of the predictions and 
# speed perfomance almost always depend on it size
seq_length = 8 # compare 2 and 32
x, y = sliding_windows(training_data, seq_length)

train_size = int(len(y) * 0.8)

x_train = x[:train_size]
y_train = y[:train_size]

x_test = x[train_size:]
y_test = y[train_size:]

print("Train data:")
print("x shape:", x_train.shape)
print("y shape:", y_train.shape)

print("\nTest data:")
print("x shape:", x_test.shape)
print("y shape:", y_test.shape)

In [ ]:
def time_series_plot(train_predict):
  data_predict = train_predict.data
  y_data_plot = y.data

  # Denormalize
  data_predict = data_predict[0] * (td_max - td_min) + td_min
  y_data_plot = y_data_plot * (td_max - td_min) + td_min 

  # Ploitting
  plt.figure(figsize=(12, 4))
  plt.axvline(x=train_size, c='r', linestyle='--')
  # shifting the curve as first y-value not correspond first value overall
  plt.plot(seq_length + np.arange(y_data_plot.shape[0]), y_data_plot)
  plt.plot(seq_length + np.arange(y_data_plot.shape[0]), data_predict)
  
  plt.title("Number of passengers per month")
  plt.ylabel("#passengers")
  plt.xlabel("Month")
  plt.xticks(labels_to_display, dataset['Month'][labels_to_display])
  
  plt.legend(['Train/Test separation', 'Real', 'Predicted'])
  plt.grid(axis='x')
  plt.show()

In [ ]:
# Define new LSTM based model

class LSTMAirTrafficPredictor(nn.Module):

    def __init__(self, input_size, hidden_size):
        # hidden_size == number of neurons 
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1) # Predict only one value

    def forward(self, x):
        out, (h, c) = self.lstm(x) 
        y = self.fc(h)
        return y

lstm =  LSTMAirTrafficPredictor(input_size=1, hidden_size=4)
input = torch.randn((108, 8, 1)) 
out = lstm(input)

print("LSTM model we use consider first input dimension as a batch dimension, output dimension logic has not changed:")
print("Input shape:".ljust(15), input.shape)
print("Output shape:".ljust(15), out.shape)

### Обучение

In [ ]:
lstm.train()

print("LSTM training process with MSE loss:")

num_epochs = 2000
learning_rate = 0.01

criterion = torch.nn.MSELoss() # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

 # Train the model
for epoch in range(num_epochs):
    y_pred = lstm(x_train) 
    optimizer.zero_grad()
    #print(outputs.shape)
    loss = criterion(y_pred, y_train.unsqueeze(0))
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
        print(f"Epoch: {epoch},".ljust(15), "loss: %1.5f" % (loss.item()))

In [ ]:
import matplotlib.pyplot as plt

lstm.eval()
train_predict = lstm(x)
labels_to_display = [i for i in range(training_data.shape[0]) if i % 12 == 0]
time_series_plot(train_predict)

### Выводы
---
1. Использование LSTM слоев при прогнозировании нестационарных временных рядов - отличное решение, т.к. нейронка способна самостоятельно выделить динамики изменениях составляющих временного ряда. Правда, в более сложных случаях (трейдинге, например, или чтении и расшифровки волн мозга) приходится прибегать к feature engineering
2. Сложность нейронной сети должна быть адекватна сложности подаваемых в нее данных. С ростом ансамбля и числа нейронов увеличивается заучивание тренировочной выборки и теряется способность к обобщению
3. Предварительный анализ цикличности в данных (если она есть) помогает понять оптимальный размер ансамбля (тут видно, что цикл, в среднем, составляет 8 интервалов)
4. Также результат может зависеть от типа скалирования, который Вы применяете. Нужно знать принципы работы скаллеров и не стесняться экспериментировать с ними. См. статьи: [Data Preprocessing with Scikit-Learn: Standardization and Scaling](https://towardsdatascience.com/data-preprocessing-with-scikit-learn-standardization-and-scaling-cfb695280412) и [Gradient Descent, the Learning Rate, and the importance of Feature Scaling](https://towardsdatascience.com/gradient-descent-the-learning-rate-and-the-importance-of-feature-scaling-6c0b416596e1)
5. При всей выгодности применения нейронных сетей, необходимо быть осторожным с автокорреляцией (см. статью [Avoiding the pitfalls](https://towardsdatascience.com/how-not-to-use-machine-learning-for-time-series-forecasting-avoiding-the-pitfalls-19f9d7adf424))

## Модификации LSTM 

Было предложено множество модификаций структуры LSTM. 



### Peephole connections
Например, введение возможности всем gates напрямую подсматривать в вектор контекста $C$. Что, безусловно, логично - сложно решать, что делать с вектором $C$ (что из него стирать, что в него добавлять, что из него брать), если видишь его только опосредованно. 

<img src="https://edunet.kea.su/repo/EduNet-content/L08/out/lstm_peepholes_connections.png" width="600px">

### Объединение forget и input gates 

Также, кажется, что правильно требовать от нейросети принимать решение о записи в/стирании из вектора $C$ одновременно. Если что-то стираем - надо что-то записать. И наоборот. 

<img src="https://edunet.kea.su/repo/EduNet-content/L08/out/lstm_join_input_and_forget_gate.png" width="600px">

### GRU (Gated reccurent unit)

Самая известная модификация LSTM - GRU. Она более компактна за счет сильных упрощений в сравнении со стандартной LSTM.

Главные изменения - объединены forget и input gates, слиты $h_t$ и $C_t$, которые в обычной LSTM только участвовали в формировании друг друга.

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/gru_basic_block.png" width="700">

In [ ]:
gru = torch.nn.GRU(input_size=4, hidden_size=3)
input = torch.randn(2, 1, 4) #seq_len, batch, input_size
h0 = torch.randn(1, 1, 3)
output, h = gru(input, h0)

print("Input shape:".ljust(15), input.shape)
print("Shape of h:".ljust(15), h.shape) # last h
print("Output shape:".ljust(15), output.shape) # seq_len = 2

Практический опыт исследователей - иногда лучше работает GRU, иногда &mdash; LSTM. Точного рецепта успеха сказать нельзя

# Sequence-to-Sequence with RNNs

Попробуем решить задачу **sequence-to-sequence**: преобразование последовательности длины $N$ в последовательность длины $T$. $T$ может быть не равно $N$.

Примеры **sequence-to-sequence** задач:
*   машинный перевод,
*   генерация ответа на вопрос,
*   генерация описания картинки или видео.

Для решения таких задач  можно использовать две **RNN**: **кодировщик** и **декодировщик**. 
* Задача **кодировщика**: обобщить информацию о **входной последовательности** $(x_1,..., x_N)$, сформировав **вектор контекста** $C$ фиксированного размера.
* Задача **декодировщик**: используя информацию из $C$, сформировать **выходную последовательность** $(y_1, ..., y_T)$.

В качестве вектора $C$ можно использовать последнее **скрытое состояние** кодировщика $h_N$.

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/seq_to_seq_with_rnn.png" width="800">

# Attention

В данном подходе мы используем один **вектор контекста** фиксированной длины $c$, в который собираем информацию со всей **входной последовательности** $(x_1,...,x_N)$.

**Входная последовательность** может содержать как единицы, так и тысячи элементов. В задаче машинного перевода **входной последовательностью** может быть:
* короткая фраза,
* абзац “Войны и мира”.

Контекст важен. Для генерации глагола в правильной форме, нужно понимать к какому существительному он относится, а для качественного перевода конца абзаца необходимо понимать, о чем шла речь в его начале. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/seq_to_seq_with_rnn.png" width="800">

При этом возникают проблемы:
- **Вектор контекста** $c$ фиксированной длины не может вместить любое количество информации, поэтому для длинных последовательностей качество будет ухудшаться. 
- На каждой итерации декодировщика **скрытое состояние** $s_t$ должно сохранять информацию о том, какие элементы **выходной последовательности** уже были сгенерированы. Если $s_t$ не способно вместить эту информацию модель может зациклиться или потерять часть **выходной последовательности**.

**Вектор контекста** $c$ и **скрытые состояния** декодировщика $s_t$ являются “бутылочными горлышками” модели.

Как решить проблему “бутылочного горлышка”?
* формировать свой контекст $c_t$ для каждого элемента **выходной последовательности** $y_t$,  
* использовать для формирования контекста $c_t$ все **скрытые состояния** кодировщика $h_i$.





## Sequence-to-Sequence with RNNs and Attention mechanism

Для формирования **векторов контекста** $(c_1, ..., c_T)$ возьмем линейную комбинацию **скрытых состояний** кодировщика $h_i$ с весами $a_{ti}$:
<center>$ c_t=\sum_{i=1}^{N}a_{ti}h_i$.</center>

Веса $a_{ti}$ указывают, какие **скрытые состояния** кодировщика $h_i$ важны для формирования элемента **выходной последовательности** $y_t$.  Они “показывают” декодировщику куда “смотреть” при генерации данного элемента. Такой механизм в нейросетях получил название **attention** (внимание).

Веса $a_{ti}$ предсказывает сама модель. Для удобства веса подбираются таким образом, чтобы их сумма для каждого **вектора контекста** $c_t$ была равна 1 (нормализация):
<center>$ \sum_{i=1}^{N}a_{ti} = 1$,</center>

<center>$  0\leqslant a_{ti} \leqslant 1$.</center>

Для этого на выходе предсказывающего веса слоя ставят **SoftMax**.

Чтобы **вектор контекста** $c_t$ содержал информацию о уже сгенеренных элементах **выходной последовательности**, значение веса до нормализации  $e_{ti}$ зависит не только от скрытого состояния кодировщика $h_i$, но и от предыдущего скрытого состояния декодировщика $s_{t-1}$.

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/seq_to_seq_with_rnn_and_attention_mechanism_step_1.png" width="1000">

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/seq_to_seq_with_rnn_and_attention_mechanism_step_2.png" width="1000">

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/seq_to_seq_with_rnn_and_attention_mechanism_step_3.png" width="700">

[Stanford University CS231n: Recurrent Neural Networks](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture10.pdf)

Давайте посмотрим, как такой подход  работает на примере перевода с английского на французский 

Как мы знаем, на каждом шаге генерируется набор весов, которые отвечают за фокусировку на том или ином месте входной последовательности. Как мы видим, английское предложение имеет иной порядок слов относительно французского. Например, в английском варианте словосочетание **European Economic Area**, в то время как во французском **zone économique européenne**. 

В английском прилагательные идут перед существительным, в то время как во французском языке наоборот. 

Таким образом, благодаря гибкости модели, мы можем обрабатывать и учитывать разный порядок слов в разных языках.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/visualize_attention_weights.png" width="700">

* Механизм внимания не обязательно должен принимать на вход последовательность. 

* Мы можем применять его, в том числе для генерации подписей для картинок. Входом в данном случае будет являться матрица признаков, которая была получена при применении сверточной сети к картинке. 

* Далее по этой матрице мы считаем веса внимания и делаем аналогично первому примеру.

Следующим шагом будет также заметить, что для достаточно длинного предложения наша модель может забыть и то, что она генерирует. Потому мы можем сделать два attention - один на представление исходного предложения, а второе - на представление того, что уже сгенерировано (что еще не сгенерировано - заменяем нулями)


## Проблема attention

Очевидно, у этого подхода есть свои минусы. 

Самый важный из них - ваша модель должна уметь предсказывать веса, с которыми складываются слова из предложения при формировании нового контекстного вектора $c$. И обычные подходы принуждают нас фиксировать размер этого вектора весов. 

За счет этого у нас возникает ограничение, от которого мы избавлялись - модель не может принимать (и, если используем attention на то, что сгенерировано, генерировать) предложения больше определенного размера. С другой стороны, ничто не мешает поставить этот максимальный размер достаточно большим, чтобы для бОльшей части предложений это работало. 

Почему это не сильный проигрыш в сравнении с LSTM - так оно все равно было ограничено в длине предложения, просто неявно, что даже хуже - лучше иметь модель, которая на длине предложения 45 падает, а не генерирует лабуду. 




## Image Captioning with RNNs and Attention

Модели, основанные на внимании (attention) намного более продвинутые, нежели обычные нейросети. Они могут концентрироваться на отдельных частях данных, что позволяет избежать зашумления представлений.

Идея состоит в том, что на каждом этапе генерации описания, нейронная сеть в разной степени обращает внимание на те или иные фрагменты изображения, соответствующие следующему слову в описании.

После обучения модели можно увидеть, что она как бы переносит своё внимание по изображению для каждого генерируемого слова.

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/image_captioning_with_rnn_and_attention_example_step_1.png" width="700">

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/image_captioning_with_rnn_and_attention_example_step_2.png" width="700">

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/image_captioning_with_rnn_and_attention_example_step_3.png" width="700">

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/image_captioning_with_rnn_and_attention_example_step_4.png" width="700">

[Stanford University CS231n: lectures](http://cs231n.stanford.edu/slides/2017/)

Посмотрим, что “привлекает внимание” нейронной сети при написании текстового описания картинки. 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/visulize_attention_map.png" width="700">

Нейронные сети, использующие механизм внимания (attention) активно применяются для решения задачи [Visual Question Answering](https://paperswithcode.com/paper/vqa-visual-question-answering). В данной задаче нейросеть должна научиться давать развернутые ответы на вопросы по изображению. Модель должна не только решать задачу классификации, но и распознавать признаки (цвет, форма, размер, количество и т.д.) предметов на изображении, различать в какой части изображения находится предмет и его положение относительно других предметов. Решение этой задачи может помочь людям с проблемами со зрением лучше ориентироваться в пространстве. 

[ссылка на исследование](https://www.reg.ru/blog/nejroset-opisyvaet-mir-nezryachim-lyudyam/) 
и еще [одна ссылка](https://www.reg.ru/blog/uchim-nejroseti-rassuzhdat-o-tom-chto-oni-vidyat/)

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/visulize_attention_map_examples.png" width="700">

## Key, query, value 

Мы можем формализовать процедуру attention через введение концепций **key**, **query** и **value**. 

К примеру, у нас есть картинка. На этой картинки у нас есть области, которые можно описать одним словом - **key**. Например - фонарь/девушка/... 

Сами эти области - это **value**, которые введенным **key** соответствуют. 

Далее нам приходит **query** - например, running. Мы можем посчитать похожесть каждого из ключей, которые у нас есть на query. 

И далее выдать информацию только по **value**, похожим на наш **query**. 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/key_query_value_example.jpg" width="800">

<center><p><em>Source: <a href="https://jonathan-hui.medium.com/nlp-bert-transformer-7f0ac397f524">NLP — BERT & Transformer</a></p> </em></center>


[Послезная статья про NLP](https://jonathan-hui.medium.com/nlp-bert-transformer-7f0ac397f524)

По сути, мы описали то, как будем делать при помощи нейронок питоновский словарь. С той разницей, что питоновский словарь может выдавать значения только для тех ключей, что в нем есть, а наш словарь выдает ответ для любого ключа-запроса, основываясь на его похожести на ключи словаря

### Key, Query, Value на примере текста 

Разобьем наше предложение на токены (в данном случае - просто слова). Получим представление каждого токена

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/key_query_value_sentence.png" width="800">

<center><p><em>Source: <a href="https://jonathan-hui.medium.com/nlp-bert-transformer-7f0ac397f524">NLP — BERT & Transformer</a></p> </em></center>

Каждое слово будет являться одновременно и key, и value. 

Теперь можно подать любой query, например - вода, и в случае, если у нас эмбединги слов хорошие, что нам "подсветятся" слова, связанные с водой

## Attention Layer

Эту идею реализует attention layer. 





Что мы, таким образом, получили? Фактически, мы получили дифференцируемый аналог словаря в Python. Только этот словарь еще умеет делать неточный поиск

У нас есть некий изначальный набор $X$ - слова в нашем предложении. 

Мы можем их преобразовать как-то, чтобы получить более удобные для дальнейшей цели $K$ - предполагается, что они лучше помогают сравнивать между собой слова. 
В простейшем случае некое линейное преобразование $K = XW_K$

Далее у нас есть значения, $V$, которые тоже получаются из $X$ путем какого-то преобразования, которое делает их более применимыми для работы модели (важные признаки и т.д.), например $V = XW_Q$



Далее к нам приходят запросы $Q$, которые находятся в том же пространстве, что и ключи. Фактически - наша нейросеть пытается вытащить из данных интересующую ее информацию. К примеру - ей нужно найти всех животных и все растения - тогда $Q = \text{\{“животные”, “растения”\}}$

Мы сравниваем эти запросы с ключами - считаем просто попарные косинусные расстояние между каждым ключом $K_i$ и запросом $Q_j$. Получаем похожесть каждого ключа на запрос, нормируем ее на корень из размерности представления нашего ключа. 

$E = \dfrac {QK^T} {\sqrt{D}} $

Получили матрицу похожестей $E$, где $E_{ij}$ - похожесть ключа $K_i$ на запрос $Q_j$

Далее мы применяем к похожестям $E$ softmax, беря его по каждому ключу отдельно (по каждому столбцу матрицы $E$). Теперь у нас похожести каждого запроса складываются в единицу. 

$A = softmax(E,\, dim=1)$

Получили для каждого запроса его "разложение" в виде ключей. 

Далее, чтобы получить значения, соответствующие каждому запросу, складываем значения с весами соответствующих ключей

$\displaystyle Y_j = \sum_iA_i V_i$

$Y = AV$

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/attention_layer.png" width="800">

### Softmax normalization 

Вы могли заметить, что в формуле для $E$ мы делим на корень из размерности представления ключа. Зачем это? 



Давайте сгенерируем вектор из многомерного нормального распределения

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 

a = np.random.normal(0, 100, size=(10000))

plt.title("Normal distribution, std = 100")
plt.ylabel("Number of samples")
plt.xlabel("Sample value")

plt.hist(a)
plt.show()

И посмотрим, что будет с распределением значений этого вектора, если к нему применить softmax

In [ ]:
from scipy.special import softmax 

plt.title("Softmax on N(0, 100)")
plt.ylabel("Softmax value")
plt.xlabel("Sample index")

plt.plot(softmax(a))
plt.show()

Почти все значения 0 и одно (наибольшее), стало 1. 
Мы получили так называемое вырожденное распределение - у него вся плотность сосредоточена в одной точке. 

И получили на абсолютно случайных данных. 

Это приведет к затуханию градиента - мы будем распространять ошибку только для 1 значения из 10000. Учиться сеть будет плохо

Но этого можно избежать - давайте просто стандартизируем наши данные 

In [ ]:
std = np.random.normal(0, 100, size=(10000))

unit_std = std / 100

plt.title("Normal distribution, std = 100")
plt.ylabel("Number of samples")
plt.xlabel("Sample value")
plt.hist(std)
plt.show()

plt.title("Normal distribution, std = 1")
plt.ylabel("Number of samples")
plt.xlabel("Sample value")
plt.hist(unit_std)
plt.show()

По сути, в распределении ничего не поменялось - только масштаб. Но теперь softmax работает нормально 

In [ ]:
plt.title("Softmax on N(0, 1)")
plt.ylabel("Softmax value")
plt.xlabel("Sample index")

plt.plot(softmax(unit_std))
plt.show()

Остается только понять, а как нормировать наши данные в нашем слое? Считать налету, наверное, не лучшая идея. 

Наш слой делает скалярное произведение между двумя векторами, предположим, нормально распределенными

In [ ]:
def statistics(dimensionality, experiments=int(10e4)):
    c = []
    for i in range(experiments):
          a = torch.normal(0, 1, size=(int(dimensionality),))
          b = torch.normal(0, 1, size=(int(dimensionality),))
          c.append(torch.dot(a, b))
        
    c = torch.Tensor(c)
    return float(c.mean()), float(c.std())

In [ ]:
import torch

means, stds = {}, {}
dims = torch.linspace(0, 100, 20)

for dim in dims:
		dim = float(dim)
		t_mean, t_std = statistics(dim)
		means[dim] = t_mean
		stds[dim] = t_std

In [ ]:
x = list(means.keys())
y = list(means.values())

plt.plot(x, y)
plt.axhline(y=0, c='r', linestyle='--')
plt.legend(['Mean value', 'Mean = 0'])
plt.title("Mean value of dot products")
plt.ylabel("Mean value")
plt.xlabel("Vector dimensionality")
plt.show()

Видим, что среднее не сильно отличается от 0 (можно показать, что в среднем оно равно 0 для произведения нормально распределенных величин). 

А вот стандартное отклонение растет. И, можно предположить, что растет оно как корень из размерности вектора. Так и есть. Потому и появляется именно такой нормировочный множитель в attention

In [ ]:
x = list(stds.keys())
y = list(stds.values())
plt.scatter(x, y)
plt.title("Std values and square distance")
plt.xlabel("Vector dimensionality")

x = np.linspace(0, 100, 10000)
plt.plot(x, x**0.5, color='r')
plt.legend(["sqrt(x)", "empirical std"])
plt.show()

### Keq, Query, Value на примере текста. Второй заход

Хорошо, у нас есть предложение, мы его разбили на токены, допустим, мы хорошо умеем кодировать эти токены. 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/key_query_value_sentence.png" width="800">

<center><p><em>Source: <a href="https://jonathan-hui.medium.com/nlp-bert-transformer-7f0ac397f524">NLP — BERT & Transformer</a></p> </em></center>

Но каждый токен закодирован на основе самого себя, и только, например, слово bank встречается в тексте два раза, и будет закодировано одинаково, хотя несет разные значения (берег и, собственно, банк). 

Как нейросети их отличать? Ну, если мы попытаемся на основе только одного токена что-то сделать - никак. 

А как улучшить кодирование каждого токена, чтобы два "банка" отличались? 

Сделаем следующее - каждый токен подадим в нейросеть **и в качестве query**. 

И будем агрегировать полученную информацию, получая токен, который будет содержать более конкретную информацию

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/key_query_value_sentence_example.png" width="800">

<center><p><em>Source: <a href="https://jonathan-hui.medium.com/nlp-bert-transformer-7f0ac397f524">NLP — BERT & Transformer</a></p> </em></center>

Например, bank, который банк, будет содержать информацию о том, что он не просто bank, а "Bank of America", который нужно найти (find).

Такой подход - когда key, query и value формируются на основе одного и того же входа нейросети называется self-attention




## Self-Attention Layer

Сделаем это все в виде слоя

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/self_attention_layer.png" width="800">

Что изменилось? 

Теперь $X$ участвует и в производстве $K$, и в производстве $V$, и в производстве $Q$. Потому и self-attention - предложение состоящее из слов $X_i$, ищет само себя в нашем дифференцируемом словаре. Почему это не приводит к тому, что мы просто получим вектор $V$?
Потому что значения $X_i$ модифицируются и $K \neq Q$.

Еще один пример проблемы с текстом, который обычные методы решали плохо

Многие из нас знают Второй Закон Робототехники

**Second Law of Robotics**


A robot must obey the orders given it by human beings except where such orders would conflict with the First Law.

Представим, что машина хочет понять, что здесь написано. И ей нужно понять, что имеется ввиду под it. 






Первым делом она формирует на основе каждого слова key и value. 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/self_attention_example_folders_step_1.png" width="800">

<center><p><em>Source: <a href="https://jalammar.github.io/illustrated-gpt2/">The Illustrated GPT-2 (Visualizing Transformer Language Models)</a></p> </em></center>

Затем она смотрит, какие ключи похожи на Query, полученный из it

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/self_attention_example_folders_step_2.png" width="800">

<center><p><em>Source: <a href="https://jalammar.github.io/illustrated-gpt2/">The Illustrated GPT-2 (Visualizing Transformer Language Models)</a></p> </em></center>

Чтобы получить, что же на самом деле значит it, машина считает взвешенную сумму values, где  веса определяются похожестью query на value

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/gpt2_value_vector_sum.png" width="800">

<center><p><em>Source: <a href="https://jalammar.github.io/illustrated-gpt2/">The Illustrated GPT-2 (Visualizing Transformer Language Models)</a></p> </em></center>

[Ссылка на оригинальную статью про GPT-2 с более подробными объяснениями как оно работает.](https://jalammar.github.io/illustrated-gpt2/)

В итоге получается, что it представляет собой "a robot" (ну и само себя). 

Ниже приведена простая реализация слоя self-attention.

In [ ]:
import torch.nn as nn

class SimplifiedSelfAttention(nn.Module): 
    def __init__(self, input_size, key_size=100, value_size=20):
        super().__init__()
        self.keyer = nn.Linear(input_size, key_size)
        self.valuer = nn.Linear(input_size, value_size)
        self.querier = nn.Linear(input_size, key_size)

        self.norm_factor = key_size ** 0.5

    def forward(self, x):
        print("Input shape:".ljust(20), x.shape)

        query = self.querier(x)
        key = self.keyer(x)
        value = self.valuer(x)

        print("Query shape:".ljust(20), query.shape)
        print("Key shape:".ljust(20), key.shape)
        print("Value shape:".ljust(20), value.shape)

        activations = torch.mm(query, key.t()) / self.norm_factor
        norm_activation = nn.functional.softmax(activations, dim=1)
        result = torch.mm(norm_activation, value)

        print("Activations shape:".ljust(20), activations.shape)
        print("Result shape:".ljust(20), result.shape)

        return result

In [ ]:
in_size = 30
data = torch.randn(10, in_size)

SA = SimplifiedSelfAttention(in_size)
result = SA(data)

### Positional encoding

Единственный возможный минус - наша нейросеть не учитывает порядка слов в предложении при составлении embedding. Это может нам мешать. Например, если в предложении два it, то они часто относятся к разным словам. Потому хотим уметь учитывать информацию о позиции. Для этого к $X$ при составлении $Q$ добавляется информация о позиции. 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/positional_encoding.png" width="700">

Делается это хитрым образом - мы берем и добавляем к каждому значению исходного вектора токенов некую комбинацию $sin$ и $cos$ с разными параметрами. 

Вектор $PE$, который мы будем добавлять к $X$ будет определяться по следующей формуле

$$PE_{pos, 2i} = \sin \left({\dfrac {pos} {10000^{2i/d}}}\right)$$

$$PE_{pos, 2i+1} = \cos \left({\dfrac {pos} {10000^{2i/d}}}\right)$$

$pos$ &mdash; это позиция токена

$d$ &mdash; количество размерностей токена

$i$ &mdash; $i$-тая размерность токена

In [ ]:
import math

class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + self.pe[:, :x.size(1)].detach() 
        return x

In [ ]:
pe = PositionalEncoding(20)
y = pe(torch.zeros(1, 100, 20)) # sequence of shape 100, every token of sequence has shape 20  

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(np.arange(100), y[0, :, 0:4].data.numpy())
plt.legend(["dim %d"%p for p in [1,2,3,4]])
plt.show()

В результате каждая позиция коддируется уникальным представлением. При этом представление позволяет легко находить слова на заданном расстоянии от исходного (у них будет одинаково значение сигнала по какой-то оси)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(np.arange(100), y[0, :, 4:8].data.numpy())
plt.legend(["dim %d"%p for p in [4, 5, 6, 7]])
plt.show()

Это помогает трансформеру достаточно уникальным образом определять каждую позицию и понимать относительное расстояние между разными токенами

### Masked Self-Attention Layer


Допустим,  у нас стоит проблема, что мы не должны видеть часть слов в предложении - например, при генерации текста (по текущим словам предсказать следующее). Например, хотим сгенерировать фразу "robot must obey orders" на основе только первого слова. 

Если мы скормим в нейронную сеть во время обучения сразу все предложение, которое хотим генерировать, то у нас в первых словах будет "протекать" информация о предыдущих.

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/queries_keys_scores_before_softmax.png" width="800">

В этом случае мы можем просто на соответствующих местах матрицы $E$ поставить минус бесконечности - тогда в эмбедингах слов, которые не должны знать о каких-то словах, информации об этих словах не будет.

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/scores_before_softmax_apply_attention_mask_masked_scores_before_softmax.png" width="800">

В результате мы сможем, после softmax "лишняя" информация не будет использоваться при генерации ответа на query

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/masked_scores_softmax_along_rows_scores.png" width="800">

Благодаря этому трюку, у нас получается обучать transfomer по-прежнему как простую single-pass нейросеть, а не "скатываться" в RNN, где у нас возникнут проблемы с градиентами и временем работы 

## Multihead Self-Attention Layer

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/multihead_self_attention_layer.png" width="700">

Давайте посмотрим на то, к каким словам предложения gave может иметь отношение. В общем случае глагол может иметь связку со многими частями предложения. Как, например, с подлежащим, так и с причастиями.

В идеале, нам бы хотелось обратить внимание функции (attention) на все эти взаимосвязи. Для этого нам просто надо поставить несколько attention слоев параллельно. В идеале каждый будет учить что-нибудь свое - аналогия с конволюциями. 

* Чтобы осуществить задуманное, вместо одного набора query, будем использовать несколько независимых наборов. 

* Причем каждый набор будет считаться уникальной матрицей. 

* Аналогично сделаем для keys и values. Количество таких наборов внутри keys, queries, values должно быть **одинаковым**. 

* Обозначим это число как h - head, далее производим аналогичные манипуляции, при этом введем в параллель h таких функций attention
* На последнем шаге мы их соединяем (конкатинируем)

* При этом можно заметить, что таком подходе на каждом шаге размерность токена будет увеличиваться (если, например, в качестве и key, и value, и query мы подаем одно и тоже предсталение токена). Если хотим сохранять управлять размерностью токена, то придется получать, по меньшей, мере value путем домножения на матрицу, размерность которой по второй оси меньше - **выполнять проекцию наших токенов в пространство меньшей размерности**

* В частности, можно подобрать размерность этого пространства таким образом, чтобы при конкатенации размерность полученного токена равнялась исходной. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/scaled_dor_product_attention_multihead.png" width="900">

## Attention is all you need

Оказывается, этот подход работает сам по себе. Не нужно добавлять никаких рекуррентных слоев - просто делаем много правильно соединенных attention layers. Полученная в результате архитектура получала название attention transformer

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/attention_is_all_you_need.png" width="800">

### Базовый блок 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/attention_transformer_basic_block.png" width="700">

Блок устроен следующим образом: входные вектора мы запускаем в блок self-attention и используем residual connection как обходной путь вокруг self-attention, затем мы их складываем. Так как обычно используют multi head self-attention, тут нам и пригождается возможность подобрать параметры так, чтобы размерности входных и выходных токенов не отличались. 

После - применяем слой нормализации, затем - слой feed forward (MLP), плюс обходной путь вокруг feed forward. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/mlp_transformer_block.png" width="900">

Складываем выход MLP-части с residual connection и выполняем нормализацию.

Нормализация везде нужна для того, чтобы нейросеть нормально обучалась. 

Заводим ли разный MLP на каждый токен (по позиции) - конечно нет. К каждому токену применяются MLP с одними и теми же весами. 


В реальных моделях выходы этих слоев стекают друг с другом и получается большая трансформер-модель. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/mlp_transformer.png" width="600">

Это пример Encoder-модели. Такова рассматриваемая далее BERT. В такой модели на выходе мы получаем некое хорошее представление исходного предложения. 

Но что если мы хотим сделать задачу перевода с одного языка на другой? Нужен еще Decoder



## Decoder 



### Первая стадия

На первой стадии то, как работает Decoder не так сильно отличается от работы Encoder. 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/decoder_transformer.jpg" width="800">

<center><p><em>Source: <a href="https://jonathan-hui.medium.com/nlp-bert-transformer-7f0ac397f524">NLP — BERT & Transformer</a></p> </em></center>

Тем не менее, есть один важный нюанс -  мы используем Masked Attention - наша нейросеть не должна видеть слов, которые еще не сгенерировала. 

На основе первого этапа у нас формируются только query - наша нейросеть на основе того, что она уже сгенерировала решает, какая информация ей нужна дальше 

### Вторая стадия

Теперь мы используем для $K$, $V$ информацию, полученную из энкодера и получаем предсказание. 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/encoder_decoder_transformer.jpg" width="800">

<center><p><em>Source: <a href="https://jonathan-hui.medium.com/nlp-bert-transformer-7f0ac397f524">NLP — BERT & Transformer</a></p> </em></center>

## Токенизация 

Перед тем как перейти к самым ярким примерам использования трансформеров, давайте разберёмся, как именно трансформеры взаимодействуют с текстом. Мы же не можем загрузить буквы, слова и предложения в нейронную сеть. Нам необходимо загружать вектора. Выше по тексту (для посимвольной генерации), эту проблему мы решали следующим образом: создавали *словарь* для букв и каждой букве присваивали индекс (a=0, b=1, c=2 ...). Давайте посмотрим, можем ли мы провернуть то же самое для предложений.

### Наивная токенизация

Рассмотрим предложение: `Я люблю Natural Language Processing (NLP). А ты?`
 
Присваивать отдельный индекс каждому предложению, кажется, совсем не оптимально. Что тогда? Мы можем разбить предложение на слова (*токенезировать предложение*) по пробелам.

In [ ]:
input = 'Я люблю Natural Language Processing (NLP). А ты?'
tokenized = input.split(' ')
print(tokenized)

Неплохо, но можно обратить внимание, что мы не учли пунктуацию. Кажется неразумным создавать отдельный индекс для каждой комбинации NLP и знаков препинания. Давайте проведём токенизацию с учётом знаков препинания

In [ ]:
import re
  
# initializing string
input = 'Я люблю Natural Language Processing (NLP). А ты?'
  
# using findall() to get all regex matches. 
res = re.findall(r'\w+|[^\s\w]+', input)
  
# printing result 
print(str(res)) 

Лучше, но представьте себе размер словаря, если мы будем токенизировать таким образом все слова в Википедии. Такой большой объём словаря приведёт к тому, что у модели будет огромный размер эмбедингов в качестве входного и выходного слоя, что приведёт к увеличению необходимой памяти. Обычно, размер словаря трансформеров не превышает 50 000 токенов. Почему бы тогда не использовать посимвольную токенезацию, как раньше?

Хотя посимвольная токенизация очень проста и значительно снижает требования к памяти, она значительно усложняет обучение модели осмысленным представлениям входных данных. Например, выучить осмысленное контекстно-независимое представление для буквы `"с"` гораздо сложнее, чем выучить контекстно-независимое представление для слова `"сегодня"`. Поэтому токенизация символов часто сопровождается потерей производительности. Чтобы получить лучшее из двух миров, трансформеры используют что-то среднее между токенизацией на уровне слов и на уровне символов, называемый токенизацией подслова (*subword tokenization*).


### *Subword Tokenization* (Токенизация подслова)

Алгоритмы токенизации подслова основываются на принципе, что часто используемые слова не должны разбиваться на более мелкие подслова, а редкие слова должны быть разложены на значимые подслова. Например, слово "*annoyingly*" может считаться редким словом и может быть разложено на "*annoying*" и "*ly*". И "*annoying*", и "*ly*" как самостоятельные подслова будут встречаться чаще, и в то же время значение слова "*annoyingly*" сохранится за счёт составного значения "*annoying*" и "*ly*".
 
Токенизация подслова позволяет модели иметь разумный объём словаря и при этом обучаться значимым контекстно-независимым представлениям. Кроме того, токенизация подслова позволяет модели обрабатывать слова, которые она никогда раньше не видела, путём разложения их на известные подслова.


Воспользуемся популярной библиотекой для токенизации от команды Hugging Face `transformers`

In [ ]:
!pip -q install transformers

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer.tokenize("I have a new GPU!")

Предложение сначала было приведено к нижнему регистру. Мы видим, что слова ["i", "have", "a", "new"] присутствуют в словаре токенизатора, а слово "gpu" - нет. Следовательно, токенизатор разбивает "gpu" на известные подслова: ["gp" и "##u"]. "##" означает, что остальная часть лексемы должна быть присоединена к предыдущей без пробела (для декодирования или обратного хода токенизации).

### Byte-Pair Encoding (BPE)


Кодирование байт-парой (BPE) опирается на претокенизатор, который разбивает обучающие данные на слова. Примером простой претокенизации может быть разбивка по пробелам. Более продвинутая предварительная токенизация включает токенизацию на основе каких-то правил.

После предварительной токинизации у нас получится набор уникальных слов и будет определена частота встречаемости каждого слова в обучающих данных. Затем BPE создаёт базовый словарь, состоящий из всех символов, которые встречаются в наборе уникальных слов, и изучает правила слияния для формирования нового символа из двух символов базового словаря. Так происходит до тех пор, пока словарный запас не достигнет желаемого размера. Обратите внимание, что желаемый объём словаря - это гиперпараметр, который необходимо определить перед обучением токенизатора.


В качестве примера предположим, что после предварительной токенизации был определён следующий набор слов, включая их частоту:

`("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)`

Видим, что базовый словарь -  `["b", "g", "h", "n", "p", "s", "u"]`. 

Разделим все слова на отдельные буквы: 

`("h" "u" "g", 10), ("p" "u" "g", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "u" "g" "s", 5)`

BPE подсчитывает частоту каждой возможной пары символов и выбирает ту пару символов, которая встречается наиболее часто. В приведённом выше примере `"h"`, за которым следует `"u"`, встречается 10 + 5 = 15 раз (10 раз в 10 вхождениях слова `"hug"`, 5 раз в 5 вхождениях слова `"hugs"`). Однако наиболее частой парой символов является `"u"`, за которой следует `"g"`, встречающаяся 10 + 5 + 5 = 20 раз в общей сложности. Таким образом, первое правило слияния, которому обучается токенизатор, - это сгруппировать все символы `"u"`, за которыми следует символ `"g"`, вместе. Затем `"ug"` добавляется в словарный запас. После этого набор слов становится следующим:

`("h" "ug", 10), ("p" "ug", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "ug" "s", 5)`

Затем BPE определяет следующую наиболее часто встречающуюся пару символов. Это `"u"`, за которым следует `"n"`, который встречается 16 раз. `"u"`, `"n"` объединяются в `"un"` и добавляются в словарь. Следующая по частоте пара символов - `"h"`, за которой следует `"ug"`, встречается 15 раз. Снова пара объединяется, и `"hug"` может быть добавлен в словарь.

На данном этапе словарный запас состоит из `["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"]`, а наш набор уникальных слов представлен как

`("hug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("hug" "s", 5)`

Если остановить обучение BPE на этом моменте, то выученные правила слияния будут применяться к новым словам (при условии, что эти новые слова не содержат символов, которых не было в базовом словаре). 

Например, слово `"bug"` будет токенизировано как `["b", "ug"]`, а `"mug"` будет токенизировано как `["<unk>", "ug"]`, поскольку символ `"m"` отсутствует в базовом словаре. Как правило, отдельные буквы, такие как `"m"`, не заменяются символом `"<unk>"`, поскольку обучающие данные обычно включают хотя бы одно вхождение каждой буквы, но это может произойти для очень специальных символов, таких как эмодзи.

Как упоминалось ранее, размер словаря, т.е. размер базового словаря + количество слияний, является гиперпараметром для выбора.

Есть и множество других форм токенизации, подробнее о них можно прочитать в [Summary of the tokenizers](https://huggingface.co/docs/transformers/tokenizer_summary).

## Примеры применений Transformer


Непосредственное применение разобранной архитектуры Encoder-Decoder для перевода текста

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/transformer_text_translation_example.png" width="800">

[The Illustrated BERT, ELMo, and co. (How NLP Cracked Transfer Learning)](https://jalammar.github.io/illustrated-bert/)

## BERT (Bidirectional Encoder Representations from Transformers )

В случае BERT используется только Encoder часть

Это нейросеть, предобученная на огромном корпусе английского текста. 

Перед ней ставили следующие задачи:

**Задача 1**

На вход дается предложение - в нем выбрано $15\%$ токенов, из которых:  
1. $80\%$ замаскированы  
2. $10\%$ заменены случайным  
3. $10\%$ оставлены без изменений




**Задача 2**

 На вход даются два предложения - сказать, идет ли второе непосредственно за первым в тексте, или нет (просто случайное предложение из корпуса)


В результате на вход подается все в таком виде:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/bert.jpg" width="800">

<center><p><em>Source: <a href="https://jonathan-hui.medium.com/nlp-bert-transformer-7f0ac397f524">NLP — BERT & Transformer</a></p> </em></center>

$CLS$ токен нужен для того, чтобы нейросети было, куда класть информацию обо всем предложении в целом. 

$SEP$ нужен просто для того, чтобы разделять два предложения (если мы подаем их два) и для того, чтобы отмечать окончание. Использование $SEP$ с двумя целями позволяет подавать в уже обученную нейросеть только одно предложение. 




### Masked Language Model

Для первой задачи используем Encoder-Decoder 

То есть у нас есть **Encoder**, который получает богатые представления и добавленный только на время обучения **Decoder** (не attention, просто MLP).

Именно Decoder отвечает за то, чтобы предсказывать пропущенные/замененные токены. Ошибка считается только по тем $15\%$ токенов, для которых могло произойти изменения, а не по всему предложению.

Как гарантируется, что модель не заменяет имевшиеся в предложение слова на другие? 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/masked_language_model.png" width="800">

<center><p><em>Source: <a href="https://jonathan-hui.medium.com/nlp-bert-transformer-7f0ac397f524">NLP — BERT & Transformer</a></p> </em></center>


### Next Sentence Prediction 

Для второй задачи - Classifier. 

При этом на вход ему подается только сам CLS токен 


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/next_sentence_prediction.png" width="800">

<center><p><em>Source: <a href="https://jonathan-hui.medium.com/nlp-bert-transformer-7f0ac397f524">NLP — BERT & Transformer</a></p> </em></center>


Обучаем модель на обеих задачах одновременно. 


### Transfer learning с BERT 

Обученную таким образом модель (оставляем только encoder), можно использовать для огромного числа других задач 


<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/transfer_learning_with_bert.jpg" width="800">

<center><p><em>Source: <a href="https://jonathan-hui.medium.com/nlp-bert-transformer-7f0ac397f524">NLP — BERT & Transformer</a></p> </em></center>

### Zero-shot learning 

Более того, есть BERT, тренированная на большом числе разных языков.

Это позволяет, например, сравнивать предложения из разных языков, хотя мы этому даже не учились. Поиграть можно [здесь](https://colab.research.google.com/github/deepmipt/dp_tutorials/blob/master/Tutorial_2_DeepPavlov_BERT_transfer_learning.ipynb#scrollTo=S1iqGcxUINyU). Веса будут грузиться ДОЛГО

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/zero_shot_learning_bert.png" width="800">

### Специализированные аналоги BERT

Более того, можно тренировать BERT под строго определенные задачи - например, анализ текстов научных статей 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/bert_specialized_analogs.png" width="800">

<center><p><em>Source: <a href="https://www.kaggle.com/code/divyadevadas/nlp-tweet-bert-roberta-eda-tensorflow-pytorch/notebook">NLP Tweet: BERT, RoBERTa, EDA,TensorFlow , Pytorch</a></p> </em></center>

## GPT (Generative Pretrained Transformer )

В случае GPT используется только Decoder часть. Но теперь во всех частях используются masked attention 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/generative_pretrained_transformer_gpt.png" width="800">

<center><p><em>Source: <a href="https://www.kaggle.com/code/abhilash1910/nlp-workshop-playing-with-transformers/notebook">NLP Workshop-Playing With Transformers</a></p> </em></center>

Это нейросеть обширно используется для, например, задачи генерации текста,

Работает она следующим образом - даем ей в качестве исходного входа какую-то затравку. 

Можно просто SOS (Start of sentence) токен, обозначающий начало предложения и больше не несущий никакой дополнительной информации


Она дает какой-то выход. Подаем этот выход нейросети как вход. 
Делаем так до тех пор, пока не надоест/не достигнем максимальной длины предложения/не встретим символ окончания генерации

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/gpt2_autoregression.gif" width="800">

<center><p><em>Source: <a href="https://jalammar.github.io/illustrated-gpt2/">The Illustrated GPT-2 (Visualizing Transformer Language Models)</a></p> </em></center>

Как ее обучали - на самом деле, тоже unsupervised. Но теперь перед моделью ставится задача предсказывать по предыдущим словам в предложении текущее.

Понятно, что такую модель можно сразу же идти и использовать для генерации, для которой она и училась 

Однако затем ее можно применять для многих других задач (опять же, путем transfer learning):


1. Классификации - подаем сразу все предложение, полученное представление используем для предсказания

2. Entailment (Определение логического следования) - даем изначальные данные, гипотезу, надо оценить, следует ли гипотеза из данных

3. Similarity - можем оценивать похожесть предложений. Так как это мы определяем порядок предложений, то, чтобы избежать неявной зависимости от порядка пропускания предложений, будем использовать результат пропускания через нейросеть обоих возможных порядков. 

4. Выбор варианта ответа на вопросы - может обучить нейросеть отвечать на вопросы с множественным выбором. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L08/out/gpt_classification_entailment_similarity.png" width="1000">

Также можно научить нейросеть отвечать на вопросы и т.д. 

Строго говоря, мы можем даже текст переводить с помощью GPT. 

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/decoder_only_transformer_translation.png" width="800">

<center><p><em>Source: <a href="https://jalammar.github.io/illustrated-gpt2/">The Illustrated GPT-2 (Visualizing Transformer Language Models)</a></p> </em></center>

Просто подаем предложение с токеном в конце, определяющим, на какой язык переводим

По аналогичной схеме можем научить нашу сеть [делать summary текста](https://arxiv.org/abs/2109.10862)

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/L08/decoder_only_summarization.png" width="800">

<center><p><em>Source: <a href="https://jalammar.github.io/illustrated-gpt2/">The Illustrated GPT-2 (Visualizing Transformer Language Models)</a></p> </em></center>

[Здесь](https://6b.eleuther.ai/) можно поиграть с open-source GPT-J

А [здесь](https://colab.research.google.com/github/kingoflolz/mesh-transformer-jax/blob/master/colab_demo.ipynb#scrollTo=e-NKauYvgTNG) - поиграть, но уже в Collab

<font size = "6">Хорошие источники 

[Про трансформеры](https://www.notion.so/Transformers-969f4b27c48147778c1e2dbda0c83ce0)

[Аннотированный трансформер](http://nlp.seas.harvard.edu/2018/04/03/attention.html)

[Код множества моделей с красивыми комментариями](https://nn.labml.ai/)

[BERT](https://medium.com/@samia.khalid/bert-explained-a-complete-guide-with-theory-and-tutorial-3ac9ebc8fa7c)

[Зоопарк BERT](https://ai.plainenglish.io/so-how-is-bert-different-ad43a42cab48)

[От NLP до Transformer](https://jonathan-hui.medium.com/nlp-bert-transformer-7f0ac397f524)

[Illustrated transformer](https://jalammar.github.io/illustrated-transformer/)

[Illustrated GPT-2](https://jalammar.github.io/illustrated-gpt2/)

[Open-source реализация GPT-3](https://arankomatsuzaki.wordpress.com/2021/06/04/gpt-j/)

[Transformer для русского языка](https://github.com/vlarine/transformers-ru)